In [9]:
import numpy as np
import math
from collections import Counter

from sklearn import datasets
from sklearn.model_selection import train_test_split

x_star = np.array([1, 5, 4, 3])
x_0 = np.array([4, 1, 2, 8])

l1_distance = 0
l2_distance = 0

for i in range(len(x_star)):
    l1_distance += abs(x_star[i] - x_0[i])
    l2_distance += (x_star[i] - x_0[i]) ** 2

l2_distance = math.sqrt(l2_distance)
print(f"L1 = {l1_distance} | L2 = {l2_distance}")

L1 = 14 | L2 = 7.3484692283495345


In [14]:
import numpy as np

x_star = np.array([1, 5, 4, 3])
x_0    = np.array([4, 1, 2, 8])

l1_distance = np.sum(np.abs(x_star - x_0))
l2_distance = np.sqrt(np.sum((x_star - x_0) ** 2))

print(f"L1 = {l1_distance} | L2 = {l2_distance:.3f}")


L1 = 14 | L2 = 7.348


In [13]:
import numpy as np
from collections import Counter

class ViiKNN:
    def __init__(self, k=3):
        # k là số lượng hàng xóm gần nhất sẽ được xét
        self.k = k
        self.X_train = None  # dữ liệu huấn luyện (features)
        self.y_train = None  # nhãn tương ứng

    def fit(self, X, y):
        """
        Với KNN không có bước training thực sự.
        Nó chỉ đơn giản là "ghi nhớ" toàn bộ dữ liệu huấn luyện.

        Parameters:
            X (numpy.ndarray(m, n)) : Dữ liệu đặc trưng (m mẫu, n đặc trưng)
            y (numpy.ndarray(m,))   : Nhãn (label) của m mẫu
        """
        self.X_train = X
        self.y_train = y

    def predict(self, X_test):
        """
        Dự đoán nhãn cho nhiều mẫu test.

        Parameters:
            X_test (numpy.ndarray(m, n)) : Dữ liệu test (m mẫu, n đặc trưng)

        Return:
            preds (list[int]) : Danh sách nhãn dự đoán
        """
        preds = []
        for i, x in enumerate(X_test):
            print(f"x_test_{i}: {x}")
            preds.append(self._predict_one(x))
        return preds
        
    # ---- Các hàm tính khoảng cách ----
    def _euclidean_distance(self, p, q):
        """
        Khoảng cách Euclidean (L2 norm)

        Parameters:
            p, q (numpy.ndarray(n,)) : 2 vector có cùng số chiều

        Return:
            scalar (float) : L2(p, q)
        """
        return np.sqrt(np.sum((p - q) ** 2))

    def _manhattan_distance(self, p, q):
        """
        Khoảng cách Manhattan (L1 norm)

        Parameters:
            p, q (numpy.ndarray(n,)) : 2 vector có cùng số chiều

        Return:
            scalar (float) : L1(p, q)
        """
        return np.sum(np.abs(p - q))

    # ---- Dự đoán cho 1 điểm dữ liệu ----
    def _predict_one(self, x):
        """
        Dự đoán nhãn cho 1 mẫu x

        Parameters:
            x (numpy.ndarray(n,)) : 1 vector đặc trưng (feature vector)

        Return:
            int : Nhãn dự đoán dựa vào k hàng xóm gần nhất
        """
        # B1: Tính khoảng cách (L2) từ x tới tất cả các mẫu trong X_train
        _distances = [self._euclidean_distance(x, x_i) for x_i in self.X_train]

        # B2: Lấy chỉ số của k khoảng cách nhỏ nhất
        k_indice = np.argsort(_distances)[:self.k]

        # B3: Lấy nhãn tương ứng với các khoảng cách nhỏ nhất
        k_nearest_labels = [self.y_train[i] for i in k_indice]
        k_nearest_distances = [_distances[i] for i in k_indice]
        
        # B4: Đếm tần suất nhãn và lấy nhãn xuất hiện nhiều nhất
        most_common = Counter(k_nearest_labels).most_common()

        # Debug/print thông tin
        print(f"Distances: {np.round(_distances, 3)}")
        print(f"The {self.k} nearest distances: {np.round(k_nearest_distances, 3)}")
        print(f"The {self.k} nearest indice: {k_indice}")
        print(f"The {self.k} nearest labels: {k_nearest_labels} | "
              f"Label value count: {most_common} | "
              f"Nearest label: {most_common[0][0]}\n")
        
        return most_common[0][0]

# Dữ liệu ví dụ
X_train = np.array([
    [1, 2],
    [2, 3],
    [3, 3],
    [6, 7],
    [7, 8]
])
y_train = np.array([0, 0, 0, 1, 1])  # Nhãn 0 hoặc 1

# Dữ liệu test
X_test = np.array([
    [3, 4],   # gần nhóm nhãn 0
    [6, 6]    # gần nhóm nhãn 1
])

# Khởi tạo và huấn luyện mô hình
model = ViiKNN(k=3)
model.fit(X_train, y_train)

# Dự đoán
preds = model.predict(X_test)
print("Predictions:", preds)


x_test_0: [3 4]
Distances: [2.828 1.414 1.    4.243 5.657]
The 3 nearest distances: [1.    1.414 2.828]
The 3 nearest indice: [2 1 0]
The 3 nearest labels: [np.int64(0), np.int64(0), np.int64(0)] | Label value count: [(np.int64(0), 3)] | Nearest label: 0

x_test_1: [6 6]
Distances: [6.403 5.    4.243 1.    2.236]
The 3 nearest distances: [1.    2.236 4.243]
The 3 nearest indice: [3 4 2]
The 3 nearest labels: [np.int64(1), np.int64(1), np.int64(0)] | Label value count: [(np.int64(1), 2), (np.int64(0), 1)] | Nearest label: 1

Predictions: [np.int64(0), np.int64(1)]
